## Analysis of 90% pangenomes

- 17 PCGs (coherent nodes with 0.01 and 0.005 cutoff during BacterialCore step)
- We will find out those functions **exclusive to each core**
- Files analyzed were created with **get_core_functions.py**

In [1]:
import os
import glob
from IPython.display import HTML, display
import csv

#### Create the reaction lists

Variables

In [2]:
# input folder
##folder_path = '/home/silvia/AAA/2022-09-13_tomate/pangenomas_tomate_2023/analisis_pans90/anotaciones_90/'
folder_path = "/home/silvia/repos/get_core_reactions/analisis_pans90.0/anotaciones_90.0"

# choose type of input
IDs = "kreac" ##bigg

# outputs
##all_reactions_file = "/home/silvia/AAA/2022-09-13_tomate/pangenomas_tomate_2023/analisis_pans90/kegg/all_reactions_to_ipath.tsv"
##unique_reactions_file = "/home/silvia/AAA/2022-09-13_tomate/pangenomas_tomate_2023/analisis_pans90/kegg/unique_reactions_to_ipath.tsv"
all_reactions_file = "/home/silvia/repos/get_core_reactions/analisis_pans90.0/kegg/all_reactions_to_ipath.tsv"
unique_reactions_file = "/home/silvia/repos/get_core_reactions/analisis_pans90.0/kegg/unique_reactions_to_ipath.tsv"
if not os.path.isdir("/home/silvia/repos/get_core_reactions/analisis_pans90.0/kegg"):
    os.makedirs("/home/silvia/repos/get_core_reactions/analisis_pans90.0/kegg")
    
# PCG table
##PCG_table = "/home/silvia/AAA/2022-09-13_tomate/tomate_bc_0.01/Tree/results.txt"
##PCG_table = "/home/silvia/repos/get_core_reactions/analisis_pans90/tomate_0.01_results.txt"
PCG_table = "/home/silvia/AAA/2023-12-06_Coherencia/all_coherent_nodes_001_AND_0005.csv"

# selected nodes
selected_nodes = ["Node119365", "Node126898", "Node147305",
           "Node15327", "Node19398", "Node21166",
           "Node28120", "Node45985", "Node52227",
           "Node53994", "Node55420",
           "Node55904"]

# colors
colors = ["Black", "Coral", "Red", "Green", "Blue", "Yellow", "Cyan", "Magenta", "Gray", "Brown", "Orange", "Purple", "Pink", "Turquoise", "Lime", "Indigo", "Teal"]
colors = ["#000000", "#FF7F50", "#FF0000", "#008000", "#0000FF", "#FFFF00", "#00FFFF", "#FF00FF", "#808080", "#A52A2A", "#FFA500", "#800080", "#FFC0CB", "#40E0D0", "#00FF00", "#4B0082", "#008080"]

# databases
##bigg_reactions_file = "/home/silvia/Apps/BiKEGG-master/bigg_models_reactions.txt"
bigg_reactions_file = "/home/silvia/repos/get_core_reactions/analisis_pans90/bigg_models_reactions.txt"
##kegg_reactions_file = "/home/silvia/rn"
kegg_reactions_file = "/home/silvia/repos/get_core_reactions/analisis_pans90/rn"

Read PCGs from PCG table and reaction descriptions from BiGG

In [3]:
taxonomy = {}

with open(PCG_table, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        if len(row) > 9:
            key = row[0]
            value = row[9]
            taxonomy[key] = value

# prepare dictionary for reaction IDs (to get descriptions)
if IDs == "bigg":
    reactions_file = bigg_reactions_file
    file_pattern = '*_bigg_reac.txt' # input for later
elif IDs == "kreac":
    reactions_file = kegg_reactions_file
    file_pattern = '*_kegg_reac.txt' 
else:
    raise SystemExit(0, "IDs must be BiGG or KEGG reactions")

description_dict = {}
with open(reactions_file, newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter="\t")
    for row in reader:
        key = row[0]
        value = row[1]
        description_dict[key] = value

#### Save all reactions (duplicates too)

This can be submitted to an iPath3/KEGG mapper map as it is.

In [4]:
# For saving everything as iPath3 input format
reac_lists = glob.glob(os.path.join(folder_path, file_pattern))

n = 0
with open(all_reactions_file, "w") as o:
    for file in reac_lists:
        node = os.path.basename(file).split('.')[0].split('_')[0]
        if node in selected_nodes:
            with open(file, 'r') as f:
                reac_list = f.read().splitlines()
                for r in reac_list:
                    r = r + "\t" + colors[n] + "\t" + "W7" + "\t" + node # W7 is the width, one of iPaths customization options
                    o.write("%s\n" % r)
        n += 1

#### Start parsing

_Unique reacs_

In [7]:
# Read in the reaction lists
reac_lists = glob.glob(os.path.join(folder_path, file_pattern))

reac_lists_data = []
group_names = []

# with open("/home/silvia/AAA/2022-09-13_tomate/pangenomas_tomate_2023/analisis_pans90/anotaciones_90/all_reactions_to_ipath.tsv", "r") as o:
for file in reac_lists:
    node = os.path.basename(file).split('.')[0].split('_')[0]
    if node in selected_nodes:
        with open(file, 'r') as f:
            reac_list = f.read().splitlines()
            reac_lists_data.append(reac_list)
            group_names.append(f"{node} ({taxonomy[node]})") # assign basename as group name
       # for r in reac_list:
       #     o.write("%s\n" % r)

# Get the set of all unique reacs
all_reacs = set()
for reac_list in reac_lists_data:
    all_reacs.update(reac_list)
    
# Identify reacs unique to each group
# Create a dictionary to store the unique reacs for each group
unique_reacs = {}
for i in range(0, len(group_names)):
    group_name = group_names[i]
    group_reacs = set(reac_lists_data[i-1])
    for j in range(0, len(group_names)):
        if j != i:
            other_group_reacs = set(reac_lists_data[j-1])
            group_reacs = group_reacs - other_group_reacs
    unique_reacs[group_name] = group_reacs

# Print out the unique reacs for each group
n = 0
with open(unique_reactions_file, "w") as o:
    for group_name, reacs in unique_reacs.items():
##        group_color = hash(group_name.split(' ')[0]) % 16777215
##        text = f"<span style='color: #{group_color:06x}'>{group_name}</span>"
        text = f"<span style='color: {colors[n]}'>{group_name}</span>"
        display(HTML(f"{text}"))
        for r in reacs:
            ## Save
            r_save = r + "\t" + colors[n] + "\t" + node
            o.write("%s\n" % r_save)
            ## Show
            if IDs == "BiGG":
                desc = r.split("_", 1)[1]
            else:
                desc = r
            try:
                display(HTML(f"<b>{r}</b>: {description_dict[desc]}"))
            except:
                print("Descriptor " + desc + " does not have a value")
        n += 1

Descriptor R05617 does not have a value


Descriptor R06858 does not have a value


Descriptor R05614 does not have a value


Descriptor R08781 does not have a value


_Shared reacs (present in no more than 2 groups):_

In [6]:
max = 3

# Read in the reac lists
reac_lists = glob.glob(os.path.join(folder_path, file_pattern))

reac_lists_data = []
group_names = []

for file in reac_lists:
    node = os.path.basename(file).split('.')[0].split('_')[0]
    if node in selected_nodes:
        with open(file, 'r') as f:
            reac_list = f.read().splitlines()
            reac_lists_data.append(reac_list)
            group_names.append(os.path.basename(file).split('.')[0].split('_')[0]) # assign basename as group name
        
# Get the set of all unique reacs
all_reacs = set()
for reac_list in reac_lists_data:
    all_reacs.update(reac_list)
    
# Identify reacs shared by no more than 2 groups
shared_reacs = {}
for reac in all_reacs:
    count = 0
    groups = []
    for i, reac_list in enumerate(reac_lists_data):
        if reac in reac_list:
            count += 1
            groups.append(group_names[i])
            if count > max:
                break
    if count <= 3:
  ##      print(groups)
        shared_reacs[reac] = sorted(groups)
    
# Print out the shared reacs and their groups
print("Shared reacs (present in no more than " + str(max) + " groups):")
##for reac, groups in sorted(shared_reacs.items(), key=lambda x: len(x[1])):
for reac, groups in shared_reacs.items():
    all_group_colors = []
    for i, group in enumerate(groups):
        group_color = hash(group) % 16777215 # reacrate a unique color for each group
        all_group_colors.append(f"<span style='color: #{group_color:06x}'>{group}</span>")
        try:
            
            display(HTML(f"<b>{reac}</b>: {all_group_colors} ({description_dict[reac]})"))
        except:
            display(HTML(f"<b>{reac}</b>: {all_group_colors}"))

Shared reacs (present in no more than 3 groups):


Create pangenomes

----


\!! Hay dos opciones: hacer el consenso de las reacciones que se incluyen en los modelos de CarveMe, o hacerlo directamente a partir de los archivos .tsv obtenidos al anotar con eggNOG. **Aquí partimos de los ficheros de anotaciones**, aunque consenso_EGG.py los llame models.

\!! Solo anotamos con eggNOG algunos genomas: aquellos asignados a las OTUs para las que tengo un hit con **Nucmer**.

-------------


1. **Nucmer**. modelado.R incluye el alineamiento con Nucmer, la creación de modelos con CarveMe. También annotate.R es capaz de llamar a Nucmer para iniciar el proceso desde el
principio

- La lista de OTUs que fueron asignadas a un genoma está en **OTU_to_genome:list.ods** <font color=blue>Hay 3438 OTUs de mis PCGs. El numero de OTUs con genomas (de estas 3438) está decente: **993**. También hay otras que quedan fuera: others, llegando a un total de 3828</font>

- Puede haber varias OTUs asignadas al mismo genoma!! Y las hay

- Also useful: **otu_to_tax.ods** (includes PCGs)

--------------

2. **Annotation**. annotate.R se encarga de la anotación funcional con eggNOG-mapper y, opcionalmente, la creación de un modelo consenso (consenso_EGG.py)

- Main script looks like:
        ```
        # ~/micro/TFM/launch_annotation_and_consensus_2022_12_14
        # RESULTSDIR=~/micro/TFM/annotations_2022_12_14

        for NODE in $(ls $INPUTDIR/models)
          ~/R-4.0.5/bin/Rscript annotate.R -g '$INPUTDIR'/models/'$NODE'/genomes --outputdir annotations_2022_12_14_'$NODE' --cores 16 --skip_consensus FALSE --dmnd_db '$PWD'/eggnog-mapper-master/d$

        ```
- Más en agenda de enero de 2023

- Guardado en ~/micro/TFM/annots_pangenomes_2022_12_14.zip

--------------

3. **consensus**. consenso_EGG.py crea una tabla de anotaciones consenso a partir de múltiples archivos de anotaciones

- Guardado también en ~/micro/TFM/annots_pangenomes_2022_12_14.zip

- Son "pans.09" en pangenomas_tomate_2023 !! 

-------------

4. Obtener listas de reacciones a partir de este pangenoma.

- Script:     anotaciones_with_GPR_associations.py.ipynb (9 enero 2023)

- Notes from Jan 2023:

```
get_core_functions
- a partir de unas anotaciones consenso para un nodo, genero:
	- un archivo con las reacciones bigg que existan en bigg_gprs. No son muchas.  --> los grafos eran con graph_SBML... pero quizá me sirven también
		- faltaría intentar salvar más. Ver celda de "pasos manuales"
	- un archivo de descripciones (a partir de EC y si no a partir de ko) --> para análisis a mano
		- podría ser que alguna entrada se quede sin descripción, salvar esas (va a mano...)
	- un archivo de kegg_reactions --> para sacar los kegg_maps
		- importante rescatar los que no tienen. --> usar kegg_kos instead, que tambien sirven para colorear
		- ok, pero por qué no los usaba antes entonces? + en un ejemplo rapido parecian superponerse...
```

- Explicación: algunas anotaciones de eggnog tienen EC, otras no; algunas tienen BiGG, otras no. Lo más consistente es KEGG en general, pero no del todo. Por eso en este script guardaba de todo un poco. Pero es importante saber que **los archivos de sufijo \_bigg_reac.txt no son exhaustivos, solo incluyen algunas reacciones de CM 1.4.1**. La lista más completa es la de KEGG Reac. Me sirve para KEGG color mapper y para iPath

- output en: pangenomas_tomate_2023/analisis_pans90/anotaciones_90/
